<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/mmd_branch/notebooks/Federated_learning_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from torchvision.datasets import CIFAR100
from torch.utils.data import Subset
import numpy as np
import random
import torch
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)

In [ ]:
clients_data = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")
clients_data.head()

,client_id,backbone,model_name,num_of_clients,path,Measurement_criteria,accuracy,loss,train_loss,splitting_method,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
0,0,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0000_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",71.760391,1.224321,1.480559,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,75.122950,2025-05-21 15:08:11,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
1,1,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0001_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",74.449878,1.023066,1.264419,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,81.579320,2025-05-21 15:09:47,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
2,2,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0002_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",68.704156,1.303382,1.526183,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,80.012801,2025-05-21 16:25:22,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
3,3,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0003_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",76.955990,0.990321,1.203102,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,78.844851,2025-05-21 16:26:50,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
4,4,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0004_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",76.405868,0.937298,1.173994,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,78.492244,2025-05-21 16:28:16,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN


In [ ]:
backbone = "dino_xcit_small_12_p16" #@param{"type":"string"}
round_number = 1 #@param{"type":"integer"}
num_of_clients = 20 #@param{"type":"integer"}
size_of_dataset = 50000 #@param{"type":"integer"}

In [ ]:

filter = (clients_data['backbone']== backbone ) & (clients_data['round_number']== round_number) & (clients_data['num_of_clients']== num_of_clients) & (clients_data['size_of_dataset']== size_of_dataset)
filtered_clients_data = clients_data[filter]

print("Number of all trained clients:", len(clients_data))
print("Number of clients after filtering:", len(filtered_clients_data))
filtered_clients_data.head()

Number of all trained clients: 21
Number of clients after filtering: 21


,client_id,backbone,model_name,num_of_clients,path,Measurement_criteria,accuracy,loss,train_loss,splitting_method,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
0,0,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0000_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",71.760391,1.224321,1.480559,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,75.122950,2025-05-21 15:08:11,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
1,1,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0001_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",74.449878,1.023066,1.264419,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,81.579320,2025-05-21 15:09:47,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
2,2,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0002_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",68.704156,1.303382,1.526183,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,80.012801,2025-05-21 16:25:22,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
3,3,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0003_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",76.955990,0.990321,1.203102,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,78.844851,2025-05-21 16:26:50,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
4,4,dino_xcit_small_12_p16,dino_xcit_small_12_p16_c0004_r0001.pth,20,/content/drive/MyDrive/MLDL_FederatedLearning/...,"accuracy,loss,train_loss",76.405868,0.937298,1.173994,i.i.d. sharing,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,78.492244,2025-05-21 16:28:16,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN


In [ ]:
#@title Dino Model
import torch
import torch.nn as nn

class DinoClassifire(nn.Module):
  def __init__(self, backbone, num_classes:int=100, device=None):
    super(DinoClassifire, self).__init__()
    self.backbone = torch.hub.load('facebookresearch/dino:main', backbone)

    #We need to freaze thhe parameters of bakbone first so we can train only on the head layer(output layer)
    for param in self.backbone.parameters():
      param.requires_grad = False

    #determine the Device
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.backbone.to(device)

    #To detect the output feature dimontion of backbone we run  Dummy forward pass
    with torch.no_grad():

      dummy_input = torch.randn(1,3,224,224).to(device)
      dummy_out = self.backbone(dummy_input)


      if isinstance(dummy_out, tuple):
        dummy_out = dummy_out[0]
      elif isinstance(dummy_out, dict):
        dummy_out = dummy_out.get("x_norm_clstoken", next(iter(dummy_out.values())))

      #If the output is 3D (B, T, D), we assume first token is the [CLS] token.
      if dummy_out.dim() == 3:
        dummy_feature = dummy_out[:,0]
      else:
        dummy_feature = dummy_out
      feature_dim = dummy_feature.shape[1]
      print("Detected feature dimontion:", feature_dim)


      #Difineing the classification Head
      self.head = nn.Linear(feature_dim, num_classes)

      #Ensure the head is trainable.
      for param in self.head.parameters():
        param.requires_grad = True

  def forward(self,x):

    #pass the input through the backbone
    features = self.backbone(x)

    if isinstance(features, tuple):
      features = features[0]
    elif isinstance(features, dict):
      features = features.get("x_norm_clstoken", next(iter(features.values())))


    # If featers are retuened as (B, T, D), use the first token
    if features.dim() == 3:
      cls_token = features[:,0]
    else:
      cls_token = features
    logits = self.head(cls_token)

    return logits

In [ ]:
def get_model(paths,sample_sizes, backbone):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = DinoClassifire(backbone=backbone, num_classes=100, device=device)
  for index in range(len(paths)):
    state_dict = torch.load(paths.iloc[index])
    model.load_state_dict(state_dict)
    model.to(device)
    yield (model,sample_sizes.iloc[index])

In [ ]:
models = list(get_model(filtered_clients_data["path"],filtered_clients_data["client_train_size"], backbone))

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main
Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main


Detected feature dimontion: 384


In [ ]:
import copy
def feb_avg(df,backbone):
  total_samples = df["client_train_size"].sum()
  global_head = None

  models = get_model(df["path"],df["client_train_size"], backbone)
  with torch.no_grad():

    for model, sample_size in models:
      if global_head is None:
        global_head = copy.deepcopy(model.head.state_dict())
        global_model = copy.deepcopy(model)
        for k in global_head.keys():
          global_head[k].zero_()

      for k in global_head.keys():
        global_head[k] += model.head.state_dict()[k] * (sample_size / total_samples)

    global_model.head.load_state_dict(global_head)

  return global_model



global_model = feb_avg(filtered_clients_data,backbone)


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main
Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main


Detected feature dimontion: 384


In [ ]:
def eveluation(model, data_loader):
  model.eval()
  correct = 0
  total = 0

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  with torch.no_grad():
    for images, lables in  data_loader:
      images, lables = images.to(device), lables.to(device)
      outputs = model(images)

      _, prediction = torch.max(outputs.data,1)

      total += lables.size(0)
      correct += (prediction == lables).sum().item()
    for i in range(len(prediction)):
      print(prediction[i], lables[i])
    return correct/ total

test_accracy = eveluation(global_model, test_loader)
print(f"evaluation accurace:{test_accracy:.2f}")


tensor(52, device='cuda:0') tensor(96, device='cuda:0')
tensor(90, device='cuda:0') tensor(90, device='cuda:0')
tensor(14, device='cuda:0') tensor(14, device='cuda:0')
tensor(80, device='cuda:0') tensor(77, device='cuda:0')
tensor(4, device='cuda:0') tensor(65, device='cuda:0')
tensor(7, device='cuda:0') tensor(7, device='cuda:0')
tensor(75, device='cuda:0') tensor(75, device='cuda:0')
tensor(93, device='cuda:0') tensor(27, device='cuda:0')
tensor(70, device='cuda:0') tensor(16, device='cuda:0')
tensor(30, device='cuda:0') tensor(30, device='cuda:0')
tensor(97, device='cuda:0') tensor(50, device='cuda:0')
tensor(0, device='cuda:0') tensor(83, device='cuda:0')
tensor(74, device='cuda:0') tensor(14, device='cuda:0')
tensor(51, device='cuda:0') tensor(51, device='cuda:0')
tensor(42, device='cuda:0') tensor(42, device='cuda:0')
tensor(70, device='cuda:0') tensor(70, device='cuda:0')
evaluation accurace:0.46
